In [49]:
%matplotlib notebook
import pprint
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)

/home/apoorv/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2885: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
sms_df = pd.read_csv('resources/SMSSpamCollection', sep='\t', header=None, names = ['status', 'msg'])
# adding the presence of words as attributes 
# -> cheap, free, offer
sms_df['spam_status'] = sms_df['status'].map(lambda x: x == 'spam')
attribute_key_list = ['cheap', 'free', 'offer']
for key in attribute_key_list:
    sms_df['attr_' + key] = sms_df['msg'].map(lambda msg: key in msg.lower())
sms_df[:3]

,status,msg,spam_status,attr_cheap,attr_free,attr_offer
0,ham,"Go until jurong point, crazy.. Available only ...",False,False,False,False
1,ham,Ok lar... Joking wif u oni...,False,False,False,False
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,True,False,True,False


In [51]:
# takes 2 columns and returns stats of how they match
# first column is the expected column
# false positives - ham marked as spam
# false negatives - spam marked as ham
def get_stats(dframe, columns):
    df_equal = dframe[dframe[columns[0]] == dframe[columns[1]]]
    df_false_positives = dframe[(dframe[columns[0]] == False) & (dframe[columns[1]] == True)]
    df_false_negatives = dframe[(dframe[columns[0]] == True) & (dframe[columns[1]] == False)]
    stats = {
        "accuracy" : len(df_equal)/len(dframe),
        "false positives" : len(df_false_positives)/len(dframe),
        "false negatives" : len(df_false_negatives)/len(dframe)
    }
    return stats

for key in attribute_key_list:
    print(key)
    pprint.pprint(get_stats(sms_df,("spam_status", "attr_"+key)))


cheap
{'accuracy': 0.8650394831299354,
 'false negatives': 0.13334529791816224,
 'false positives': 0.001615218951902369}
free
{'accuracy': 0.8898061737257718,
 'false negatives': 0.09834888729361091,
 'false positives': 0.011844938980617373}
offer
{'accuracy': 0.8713208901651113,
 'false negatives': 0.12724335965541997,
 'false positives': 0.0014357501794687725}
